In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [2]:
PATH='data/nietzsche/'

In [3]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

corpus length: 600893


In [4]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

Map from chars to indices and back again

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [9]:
idx = [char_indices[c] for c in text]

In [10]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [11]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [12]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

Our inputs

In [13]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

Our output

In [14]:
y = np.stack(c4_dat[:-2])

The first 4 inputs and outputs

In [14]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [15]:
y[:4]

array([30, 29,  1, 40])

In [16]:
x1.shape, y.shape

((200295,), (200295,))

### Create and train model

Pick a size for our hidden state

In [15]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [16]:
n_fac = 42

In [30]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [20]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [21]:
m = Char3Model(vocab_size, n_fac).cuda()

In [22]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [191]:
opt = optim.Adam(m.parameters(), 1e-2)

In [192]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       2.09627  6.52849]                                 



In [193]:
set_lrs(opt, 0.001)

In [194]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.84525  6.52312]                                 



### Test model

In [195]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [196]:
get_next('y. ')

'T'

In [197]:
get_next('ppl')

'e'

In [198]:
get_next(' th')

'e'

In [199]:
get_next('and')

' '

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [19]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [20]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs-1)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [21]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs-1)]

In [22]:
xs = np.stack(c_in_dat, axis=0)

In [23]:
xs.shape

(600884, 8)

In [24]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [25]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [42, 29, 30, 25, 27, 29,  1,  1],
       [29, 30, 25, 27, 29,  1,  1,  1],
       [30, 25, 27, 29,  1,  1,  1, 43],
       [25, 27, 29,  1,  1,  1, 43, 45],
       [27, 29,  1,  1,  1, 43, 45, 40],
       [29,  1,  1,  1, 43, 45, 40, 40],
       [ 1,  1,  1, 43, 45, 40, 40, 39]])

...and this is the next character after each sequence.

In [26]:
y[:cs]

array([ 1,  1, 43, 45, 40, 40, 39, 43])

### Create and train model

In [27]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [28]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [158]:
class CharLoopModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h))

In [159]:
m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [160]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       2.01881  2.0294 ]                                 



In [161]:
set_lrs(opt, 0.001)

In [162]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.      1.7161  1.7246]                                    



In [163]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h))

In [164]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [165]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [166]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.80699  1.7688 ]                                



In [167]:
set_lrs(opt, 1e-4)

In [168]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.69154  1.68602]                                 



### Test model

In [46]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [47]:
get_next('for thos')

'e'

In [48]:
get_next('part of ')

't'

In [49]:
get_next('queens a')

'n'

## RNN with pytorch

In [169]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]))

In [170]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [171]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [172]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([8, 512, 42])

In [173]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [175]:
t = m(*V(xs)); t.size()

torch.Size([512, 85])

In [176]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.86162  1.84379]                                 
[ 1.       1.66857  1.66926]                                 
[ 2.       1.5879   1.60001]                                 
[ 3.       1.55005  1.55722]                                 



In [177]:
set_lrs(opt, 1e-4)

In [178]:
fit(m, md, 2, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.45798  1.5094 ]                                 
[ 1.       1.45333  1.50419]                                 



### Test model

In [179]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [180]:
get_next('for thos')

'e'

In [181]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [182]:
get_next_n('for thos', 40)

'for those of the same to the same to the same to'

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [29]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [30]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [31]:
xs = np.stack(c_in_dat)
xs.shape

(75111, 8)

In [32]:
ys = np.stack(c_out_dat)
ys.shape

(75111, 8)

In [33]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [ 1,  1, 43, 45, 40, 40, 39, 43],
       [33, 38, 31,  2, 73, 61, 54, 73],
       [ 2, 44, 71, 74, 73, 61,  2, 62],
       [72,  2, 54,  2, 76, 68, 66, 54],
       [67,  9,  9, 76, 61, 54, 73,  2],
       [73, 61, 58, 67, 24,  2, 33, 72],
       [ 2, 73, 61, 58, 71, 58,  2, 67]])

In [34]:
ys[:cs,:cs]

array([[42, 29, 30, 25, 27, 29,  1,  1],
       [ 1, 43, 45, 40, 40, 39, 43, 33],
       [38, 31,  2, 73, 61, 54, 73,  2],
       [44, 71, 74, 73, 61,  2, 62, 72],
       [ 2, 54,  2, 76, 68, 66, 54, 67],
       [ 9,  9, 76, 61, 54, 73,  2, 73],
       [61, 58, 67, 24,  2, 33, 72,  2],
       [73, 61, 58, 71, 58,  2, 67, 68]])

### Create and train model

In [35]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [36]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [136]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp))

In [137]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [138]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [139]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [140]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.      1.1164  0.9535]                                   
[ 1.       0.83636  0.76285]                                 
[ 2.       0.71097  0.68187]                                
[ 3.       0.64594  0.63435]                                 



In [141]:
set_lrs(opt, 1e-4)

In [142]:
fit(m, md, 1, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       0.59991  0.61257]                                 



### Identity init!

In [145]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [146]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [147]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       0.80381  0.7284 ]                                 
[ 1.       0.66663  0.63739]                                 
[ 2.       0.61271  0.6188 ]                                 
[ 3.       0.59271  0.60541]                                 



In [148]:
set_lrs(opt, 1e-3)

In [149]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       0.5156   0.53407]                                 
[ 1.       0.49554  0.52318]                                
[ 2.       0.48124  0.51721]                                 
[ 3.       0.47085  0.51224]                                 



In [150]:
set_lrs(opt, 1e-4)

In [151]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       0.4549   0.50927]                                 
[ 1.       0.45277  0.50849]                                 
[ 2.       0.45234  0.50804]                                 
[ 3.       0.45165  0.50758]                                 



## Stateful model

### Setup

In [156]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

In [157]:
PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt  trn/  val/


In [158]:
TEXT = data.Field(lower=True, tokenize=list)

In [159]:
bs=64; bptt=8; n_fac=42; n_hidden=256

In [160]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

In [161]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(963, 56, 1, 493747)

In [162]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.h = V(torch.zeros(1, bs, n_hidden))
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(cs)
        outp,h = self.rnn(inp, self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp)).view(-1,vocab_size)

In [172]:
m = CharSeqStatefulRnn(vocab_size, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [173]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden));

In [167]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.13513  1.08475]                                 



In [174]:
set_lrs(opt, 1e-3)

In [175]:
fit(m, md, 2, opt, F.nll_loss)

A Jupyter Widget

[ 0.       0.96742  0.96103]                                  
[ 1.       0.88555  0.8695 ]                                  



In [176]:
fit(m, md, 2, opt, F.nll_loss)

A Jupyter Widget

[ 0.       0.87285  0.881  ]                                  
[ 1.       0.87575  0.76261]                                  



In [177]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       0.80812  0.82194]                                  
[ 1.       0.85778  0.80804]                                  
[ 2.       0.79613  0.73514]                                  
[ 3.       0.827    0.78421]                                  



In [178]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       0.73793  0.76126]                                  
[ 1.       0.74701  0.7797 ]                                  
[ 2.       0.69897  0.75077]                                  
[ 3.       0.69903  0.73123]                                  



In [179]:
set_lrs(opt, 1e-4)

In [180]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       0.65736  0.72496]                                  
[ 1.       0.66048  0.74494]                                  
[ 2.       0.68257  0.75462]                                  
[ 3.       0.69551  0.76302]                                  



### Test

In [181]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
#     print(p)
    r = np.argmax(to_np(p), axis=1)
#     print(r.shape)
    return TEXT.vocab.itos[r[-1]]

In [182]:
get_next('for thos')

'k'

In [181]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [182]:
get_next_n('for thos', 40)

'for those of the same to the same to the same to'